### Build a Question Answering application over a Graph Database

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

print(f"Connecting to Neo4j at: {NEO4J_URI}")

Connecting to Neo4j at: neo4j+s://0bacf39e.databases.neo4j.io


In [7]:
from langchain_community.graphs import Neo4jGraph
import warnings
warnings.filterwarnings("ignore")
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    refresh_schema=False  #  prevents ValueError
)
graph

In [9]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [10]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n"

In [16]:
def safe_query(self, query: str, params=None):
    with self._driver.session(database=self._database) as session:
        result = session.run(query, params or {})
        return [record.data() for record in result]

from langchain_community.graphs import Neo4jGraph
Neo4jGraph.query = safe_query


In [17]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, POB: STRING, DOB: STRING}
Company {name: STRING}
empolyee {name: STRING}
enterpreneur {name: STRING, POB: STRING, DOB: STRING}
Country {name: STRING}
Person {name: STRING, born: STRING}
Movie {name: STRING, release: STRING, id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
User {name: STRING, city: STRING, age: INTEGER}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [34]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY2")

In [28]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11b9ac890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11b9aca40>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x1167515b0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11b9ac890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x11b9aca40>, model_name='gemma2-9b-it', model

In [30]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': "I don't know the answer. \n"}

In [31]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name: "Casino"})<-[:ACTED_IN]-(a:Person)RETURN a.name

Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [32]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (p:Person) RETURN count(p) AS count
Full Context:
[{'count': 1242}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1242 artists. \n'}

In [33]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN count(m) 

Full Context:
[{'count(m)': 2}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in',
 'result': "I don't know the answer. \n"}